This exercise consists in analyzing a dataset containg timing information from a series of Time-to-Digital-Converters (TDC) implemented in a pair of FPGAs. Each measurement (i.e. each row of the input file) consists of a flag that specifies the type of message ('HEAD', which in this case is always 1), two addresses of the TDC providing the signal ('FPGA' and 'TDC_CHANNEL'), and the timing information ('ORBIT_CNT', 'BX_COUNTER', and 'TDC_MEAS'). Each TDC count corresponds to 25/30 ns, whereas a unit of BX_COUNTER corresponds to 25 ns, and the ORBIT_CNT is increased every 'x' BX_COUNTER. This allows to store the time in a similar way to hours, minutes and seconds.

In [ ]:
# If haven't downloaded it yet, please get the data file with wget
#!wget https://www.dropbox.com/s/xvjzaxzz3ysphme/data_000637.txt -P ./data/

1\. Create a Pandas DataFrame reading N rows of the `data/data_000637.txt` dataset. Choose N to be smaller than or equal to the maximum number of rows and larger that 10k (check the documentation).

In [2]:
import pandas as pd
import numpy as np

N=100000 #number of rows readed in the file
data_000637 = 'data/data_000637.txt'
# read and store the data from the file and create a dataframe
df = pd.read_csv(data_000637,nrows=N)

df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS
0,1,0,123,3869200167,2374,26
1,1,0,124,3869200167,2374,27
2,1,0,63,3869200167,2553,28
3,1,0,64,3869200167,2558,19
4,1,0,64,3869200167,2760,25
...,...,...,...,...,...,...
99995,1,0,64,3869201161,2378,29
99996,1,0,70,3869201161,2472,26
99997,1,0,58,3869201161,2558,0
99998,1,0,57,3869201161,2561,23


2\. Estimate the number of BX in a ORBIT (the value 'x').

In [4]:
# the number of BX in a ORBIT (the value 'x') corresponds to the maximum value of BX_Counter
x=df['BX_COUNTER'].max()
print("x=",x)

x= 3563


3\. Find out the duration of the data taking in hours, minutes and seconds. You can either make an estimate based on the fraction of the measurements (rows) you read, or perform this check precisely by reading the whole dataset.

In [8]:
#  duration = number of ORBIT_CNT * x * 25ns
duration = len(df.groupby('ORBIT_CNT').count())* x * 25/10**6
print("duration (ms) = ",duration)

duration (ms) =  88.629625


4\. Create a new column with the absolute time in ns (as a combination of the other three columns with timing information) since the beginning of the data acquisition.

In [6]:
time_0 = df.at[0,'ORBIT_CNT']*x*25 + df.at[0,'BX_COUNTER']*25 + df.at[0,'TDC_MEAS']*(25/30)

# New column : Absolute time in nanosecond
df['time in ns'] = df['ORBIT_CNT']*x*25 + df['BX_COUNTER']*25 + df['TDC_MEAS']*(25/30) - time_0
df

,HEAD,FPGA,TDC_CHANNEL,ORBIT_CNT,BX_COUNTER,TDC_MEAS,time in ns
0,1,0,123,3869200167,2374,26,0.000000e+00
1,1,0,124,3869200167,2374,27,8.125000e-01
2,1,0,63,3869200167,2553,28,4.476625e+03
3,1,0,64,3869200167,2558,19,4.594125e+03
4,1,0,64,3869200167,2760,25,9.649125e+03
...,...,...,...,...,...,...,...
99995,1,0,64,3869201161,2378,29,8.854065e+07
99996,1,0,70,3869201161,2472,26,8.854300e+07
99997,1,0,58,3869201161,2558,0,8.854513e+07
99998,1,0,57,3869201161,2561,23,8.854522e+07


5\. Use the `.groupby()` method to find out the noisy channels, i.e. the TDC channels with most counts (print to screen the top 3 and the corresponding counts)

In [21]:
#Count all the values for each TDC channel
TDC_channel_count = df.groupby('TDC_CHANNEL').count()

noisy_channel = TDC_channel_count.sort_values(by=['TDC_MEAS'],ascending = False)[0:3]

noisy_channel

,HEAD,FPGA,ORBIT_CNT,BX_COUNTER,TDC_MEAS,ABS_TIME
TDC_CHANNEL,,,,,,
139,8320,8320,8320,8320,8320,8320
64,5027,5027,5027,5027,5027,5027
63,4917,4917,4917,4917,4917,4917


6\. Count the number of non-empty orbits (i.e. the number of orbits with at least one hit). Count also the number of unique orbits with at least one measurement from TDC_CHANNEL=139.

In [5]:
non_empty_orbits = len(df.groupby('ORBIT_CNT').size())
print("Non-empty orbits :", non_empty_orbits)

orbits_TDC_139 = len(df[df['TDC_CHANNEL'] == 139].groupby('ORBIT_CNT').size())
print("Orbits with at least one measurement from TDC_CHANNEL 139 :", orbits_TDC_139)

Non-empty orbits : 995
Orbits with at least one measurement from TDC_CHANNEL 139 : 994


7\. **Optional:** Make two occupancy plots (one for each FPGA), i.e. plot the number of counts per TDC channel